In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [2]:
from grader import score

# Capstone: The happiest places.
## Doing NLP, counting positive words in Reviews
#### See Yelp blog -- > 
We will take the no. of positive words and turn them into a ranking
for places, based on where to take sad people to cheer them up.
(Assuming the most happy = the more effective. Obv will need to
troubleshoot later.)

It's good to use Rich's NLP Adv ML lecture (5.1, 1/29/18) to review
strategy for counting the # of positive words in yelp review, to
rank for our users to consider for outings with sad friends.

Things to consider:

1) Use Spacy! Powerful parser. Can download inline (see beginning)

2) Choose bw a counting vector (counts of each word) or hashing vector
(also counts but can't trace words)

3) Feature engineering. Filtering stop words, which spacy and scikit do. And filtering uncommon words (TF-IDF) 

# NLP: Analyzing Review Text

Unstructured data makes up the vast majority of data.  This is a basic intro to handling unstructured data.  Our objective is to be able to extract the sentiment (positive or negative) from review text.  We will do this from Yelp review data.

The first three questions task you to build models, of increasing complexity, to predict the rating of a review from its text.  These models will be assessed based on the root mean squared error of the number of stars predicted.  There is a reference solution (which should not be too hard to beat) that defines the score of 1.

The final question asks only for the result of a calculation, and your results will be compared directly to those of a reference solution.

## A note on scoring
It **is** possible to score >1 on these questions. This indicates that you've beaten our reference model - we compare our model's score on a test set to your score on a test set. See how high you can go!

## Download and parse the data

To start, let's download the dataset from Amazon S3:

In [3]:
!aws s3 sync s3://dataincubator-course/mldata/ . --exclude '*' --include 'yelp_train_academic_dataset_review.json.gz'

download: s3://dataincubator-course/mldata/yelp_train_academic_dataset_review.json.gz to ./yelp_train_academic_dataset_review.json.gz


The training data are a series of JSON objects, in a gzipped file. Python supports gzipped files natively: [gzip.open](https://docs.python.org/2/library/gzip.html) has the same interface as `open`, but handles `.gz` files automatically.

The built-in json package has a `loads()` function that converts a JSON string into a Python dictionary.  We could call that once for each row of the file. [ujson](http://docs.micropython.org/en/latest/library/ujson.html) has the same interface as the built-in `json` library, but is *substantially* faster (at the cost of non-robust handling of malformed json).  We will use that inside a list comprehension to get a list of dictionaries:

In [4]:
import gzip
import ujson as json

with gzip.open('yelp_train_academic_dataset_review.json.gz') as f:
    data = [json.loads(line) for line in f]

If you're having trouble with running out of memory, you can do quite well with a 5% or 10% subsample of the data.  A quick way to take a random subsample is to run the following load instead:

In [5]:
import gzip
import ujson as json
import random

#Fraction to subsample, needs to be between 0 and 1
subsample = 0.1

with gzip.open('yelp_train_academic_dataset_review.json.gz') as f:
    data = [json.loads(line) for line in f if random.random() < subsample]

Scikit Learn will want the labels in a separate data structure, so let's pull those out now.

In [6]:
stars = [row['stars'] for row in data]

### Notes:
1. [Pandas](http://pandas.pydata.org/) is able to read JSON text directly.  Use the `read_json()` function with the `lines=True` keyword argument.  While the rest of this notebook will assume you are using a list of dictionaries, you can complete it with dataframes, if you so desire. Some of the example code will need to be modified in this case.

2. There are obvious miscodings in the data.  There is no need to try to correct them.

## Building models

For the first three questions, you will need to build and train an estimator to predict the star rating from the text of a review.  We recommend building a pipeline out of transformers and estimators provided by Scikit Learn.  You can decide whether these pipelines should take full review objects or just their text as input to the `fit()` and `predict()` methods, but it does pay to be consistent.

You may find it useful to serialize the trained models to disk.  This will allow you to reload the models after restarting the notebook, without needing to retrain them.  We recommend using the [`dill` library](https://pypi.python.org/pypi/dill) for this (although the [`joblib` library](http://scikit-learn.org/stable/modules/model_persistence.html) also works).  Use
```python
dill.dump(estimator, open('estimator.dill', 'w'))
```
to serialize the object `estimator` to the file `estimator.dill`.  If you have trouble with this, try setting the `recurse=True` keyword args in the call of `dill.dump()`.  The estimator can be deserialized with
```python
estimator = dill.load(open('estimator.dill', 'r'))
```

You may run into trouble with the size of your models and Digital Ocean's memory limit. This is a major concern in real-world applications. Your production environment will likely not be that different from Digital Ocean and being able to deploy there is important. Think about what information the different stages of
your pipeline need and how you can reduce the memory footprint.

Additionally, you may notice that your serialized models are very large and take a long time to load.  Some hints to reduce their size:

- If you are using `GridSearchCV` to find the optimal values of hyperparameters (and you should be), the resultant object will contain many copies of the estimator that aren't needed any more.  Instead of serializing the whole `GridSearchCV`, serialize just the estimator with the correct hyperparameters.  This can be accessed through the `.best_estimator_` attribute of the `GridSearchCV` object.  Alternatively, the `.best_params_` attribute gives the best values of the hyperparameters.

- The `CountVectorizer` keeps track of all words that were excluded from vectorization in its `.stop_words_` attribute.  This can be interesting to examine, but isn't needed for predictions.  Set this attribute to the empty list before serializing it to save disk space.

In [8]:
import dill
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import base
from sklearn.linear_model import LinearRegression, Ridge
from tqdm import tqdm_notebook

# Questions

Each of the "model" questions asks you to create a function that models the number of stars given in a review from the review text.  It will be passed a list of dictionaries.  Each of these will have the same format as the JSON objects you've just read in.  This function should return a list of numbers of the same length, giving the predicted star ratings.

This function is passed to the `score()` function, which will receive input from the grader, run your function with that input, report the results back to the grader, and print out the score the grader returned.  Depending on how you constructed your estimator, you may be able to pass the predict method directly to the `score()` function.  If not, you will need to write a small wrapper function to mediate the data types.

## bag_of_words_model
Build a linear model predicting the star rating based on the count of the words in each document (bag-of-words model).  Use a [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) or [HashingVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html#sklearn.feature_extraction.text.HashingVectorizer) to produce a feature matrix giving the counts of each word in each review.  Feed this in to linear model, such as `Ridge` or `SGDRegressor`, to predict the number of stars from each review.

**Hints**:
1. Don't forget to use tokenization!  This is important for good performance but it is also the most expensive step.  Try vectorizing as a first initial step and then running grid-serach and cross-validation only on of this pre-processed data.  `CountVectorizer` has to memorize the mapping between words and the index to which it is assigned.  This is linear in the size of the vocabulary.  The `HashingVectorizer` does not have to remember this mapping and will lead to much smaller models.

 ```python
 from sklearn.feature_extraction.text import CountVectorizer

 text = [row['text'] for row in data]
 X = CountVectorizer().fit_transform(text)

 # Now, this can be run with many different parameters
 # without needing to retrain the vectorizer:
 model.fit(X, stars, hyperparameter=something)
 ```

2. Try choosing different values for `min_df` (minimum document frequency cutoff) and `max_df` in `CountVectorizer`.  Setting `min_df` to zero admits rare words which might only appear once in the entire corpus.  This is both prone to overfitting and makes your data unmanageably large.  Don't forget to use cross-validation or to select the right value.  Notice that `HashingVectorizer` doesn't support `min_df`  and `max_df`.  However, it's not hard to roll your own transformer that solves for these.

3. Try using [LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) or [RidgeCV](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html#sklearn.linear_model.RidgeCV).  If the memory footprint is too big, try switching to [Stochastic Gradient Descent](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#sklearn.linear_model.SGDRegressor) You might find that even ordinary linear regression fails due to the data size.  Don't forget to use [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) to determine the regularization parameter!  How do the regularization parameter `alpha` and the values of `min_df` and `max_df` from `CountVectorizer` change the answer?

4. You will likely pick up several hyperparameters between the tokenization step and the regularization of the estimator.  While is is more strictly correct to do a grid search over all of them at once, this can take a long time. Quite often, doing a grid search over a single hyperparameter at a time can produce similar results.  Alternatively, the grid search may be done over a smaller subset of the data, as long as it is representative of the whole.

5. Finally, assemble a pipeline that will transform the data from records all the way to predictions.  This will allow you to submit its predict method to the grader for scoring.

In [9]:
class ReviewTextExtractor(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col_names):
        self.col_names = col_names # We will need these in transform()
        #print self.col_names
    def fit(self, X, y=None):
        # This transformer doesn't need to learn anything about the data,
        # so it can just return self without any further processing
        return self
    
    def transform(self, X):
        # Return an array with the same number of rows as X and one
        # column for each in self.col_names
        self.transformed = [[row[col_name] for col_name in self.col_names][0] for row in X]
        return self.transformed

In [10]:
from sklearn.pipeline import Pipeline

bag_of_words_est = Pipeline([
    # Column selector (remember the ML project?)
    ("Column Selector", ReviewTextExtractor(['text'])),
    # Vectorizer
    ("Hash Vectorizer", HashingVectorizer()),
    # Frequency filter (if necessary)
    # Regressor
    ("Regressor", Ridge())
])
bag_of_words_est.fit(data, stars)

Pipeline(steps=[('Column Selector', ReviewTextExtractor(col_names=['text'])), ('Hash Vectorizer', HashingVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
         dtype=<type 'numpy.float64'>, encoding=u'utf-8', input=u'content',
         lowercase=True, n_features=1048576, ngram_range=(1, ...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

In [7]:
from sklearn.pipeline import Pipeline

bag_of_words_est = Pipeline([
    # Column selector (remember the ML project?)
    # Vectorizer
    # Frequency filter (if necessary)
    # Regressor
])
bag_of_words_est.fit(data, stars)

ValueError: need more than 0 values to unpack

In [11]:
score('nlp__bag_of_words_model', bag_of_words_est.predict)

Your score:  1.37177369109


## normalized_model
Normalization is key for good linear regression. Previously, we used the count as the normalization scheme.  Add in a normalization transformer to your pipeline to improve the score.  Try some of these:

1. You can use the "does this word present in this document" as a normalization scheme, which means the values are always 1 or 0.  So we give no additional weight to the presence of the word multiple times.

2. Try using the log of the number of counts (or more precisely, $log(x+1)$). This is often used because we want the repeated presence of a word to count for more but not have that effect tapper off.

3. [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) is a common normalization scheme used in text processing.  Use the [TFIDFTransformer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer). There are options for using `idf` and taking the logarithm of `tf`.  Do these significantly affect the result?

Finally, if you can't decide which one is better, don't forget that you can combine models with a linear regression.

In [16]:
score('nlp__normalized_model', bag_of_words_est.predict)

Your score:  1.26663390916


## bigram_model
In a bigram model, we'll consider both single words and pairs of consecutive words that appear.  This is going to be a much higher dimensional problem (large $p$) so you should be careful about overfitting.

Sometimes, reducing the dimension can be useful.  Because we are dealing with a sparse matrix, we have to use [TruncatedSVD](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD).  If we reduce the dimensions, we can use a more sophisticated models than linear ones.

As before, memory problems can crop up due to the engineering constraints. Playing with the number of features, using the `HashingVectorizer`, incorporating `min_df` and `max_df` limits, and handling stop-words in some way are all methods of addressing this issue. If you are using `CountVectorizer`, it is possible to run it with a fixed vocabulary (based on a training run, for instance). Check the documentation.

**A side note on multi-stage model evaluation:** When your model consists of a pipeline with several stages, it can be worthwhile to evaluate which parts of the pipeline have the greatest impact on the overall accuracy (or other metric) of the model. This allows you to focus your efforts on improving the important algorithms, and leaving the rest "good enough".

One way to accomplish this is through ceiling analysis, which can be useful when you have a training set with ground truth values at each stage. Let's say you're training a model to extract image captions from websites and return a list of names that were in the caption. Your overall accuracy at some point reaches 70%. You can try manually giving the model what you know are the correct image captions from the training set, and see how the accuracy improves (maybe up to 75%). Alternatively, giving the model the perfect name parsing for each caption increases accuracy to 90%. This indicates that the name parsing is a much more promising target for further work, and the caption extraction is a relatively smaller factor in the overall performance.

If you don't know the right answers at different stages of the pipeline, you can still evaluate how important different parts of the model are to its performance by changing or removing certain steps while keeping everything else constant. You might try this kind of analysis to determine how important adding stopwords and stemming to your NLP model actually is, and how that importance changes with parameters like the number of features.

In [13]:
bigram_est = Pipeline([
    # Column selector (remember the ML project?)
    ("Column Selector", ReviewTextExtractor(['text'])),
    # Vectorizer
    ("Hash Vectorizer", HashingVectorizer(ngram_range = (1,2))),
    # Frequency filter (if necessary)
    # Regressor
    ("Regressor", Ridge())
])

In [14]:
bigram_est.fit(data, stars)

Pipeline(steps=[('Column Selector', ReviewTextExtractor(col_names=['text'])), ('Hash Vectorizer', HashingVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
         dtype=<type 'numpy.float64'>, encoding=u'utf-8', input=u'content',
         lowercase=True, n_features=1048576, ngram_range=(1, ...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

In [15]:
score('nlp__bigram_model', bigram_est.predict)

Your score:  1.37559109197


## food_bigrams
Look over all reviews of restaurants.  You can determine which businesses are restaurants by looking in the `yelp_train_academic_dataset_business.json.gz` file from the ml project or downloaded below.

In [17]:
!aws s3 sync s3://dataincubator-course/mldata/ . --exclude '*' --include 'yelp_train_academic_dataset_business.json.gz'

download: s3://dataincubator-course/mldata/yelp_train_academic_dataset_business.json.gz to ./yelp_train_academic_dataset_business.json.gz


In [18]:
with gzip.open('yelp_train_academic_dataset_business.json.gz') as f:
    business_data = [json.loads(line) for line in f]

Each row of this file corresponds to a single business.  The category key gives a list of categories for each; take all where "Restaurants" appears.

In [19]:
restaurant_ids = [row[u'business_id'] for row in business_data if u'Restaurants' in row[u'categories']]

In [20]:
assert len(restaurant_ids) == 12876

The "business_id" here is the same as in the review data.  Use this to extract the review text for all reviews of restaurants.

In [21]:
restaurant_reviews = []
for row in tqdm_notebook(data):
    if row[u'business_id'] in restaurant_ids:
        restaurant_reviews.append(row[u'text'])

HBox(children=(IntProgress(value=0, max=101265), HTML(value=u'')))

In [22]:
assert len(restaurant_reviews) == 574278

AssertionError: 

In [24]:
dill.dump(restaurant_reviews, open("Rest_Reviews.pkl", "w"))

In [25]:
dill.dump(restaurant_ids, open("Rest_IDs.pkl", "w"))

In [26]:
dill.dump(restaurant_starstars, open("Rest_stars.pkl", "w"))

NameError: name 'restaurant_starstars' is not defined

We want to find collocations --- that is, bigrams that are "special" and appear more often than you'd expect from chance. We can think of the corpus as defining an empirical distribution over all ngrams.  We can find word pairs that are unlikely to occur consecutively based on the underlying probability of their words. Mathematically, if $p(w)$ be the probability of a word $w$ and $p(w_1 w_2)$ is the probability of the bigram $w_1 w_2$, then we want to look at word pairs $w_1 w_2$ where the statistic

  $$ \frac{p(w_1 w_2)}{p(w_1) p(w_2)} $$

is high.  Return the top 100 (mostly food) bigrams with this statistic with the 'right' prior factor (see below).

Estimating the probabilities is simply a matter of counting, and there are number of approaches that will work.  One is to use one of the tokenizers to count up how many times each word and each bigram appears in each review, and then sum those up over all reviews.  You might want to know that te `CountVectorizer` has a `.get_feature_names()` method which gives the string associated with each column.  (Question for thought: Why doesn't the `HashingVectorizer` have a similar method?)

*Questions:* This statistic is a ratio and problematic when the denominator is small.  We can fix this by applying Bayesian smoothing to $p(w)$ (i.e. mixing the empirical distribution with the uniform distribution over the vocabulary).

1. How does changing this smoothing parameter affect the word pairs you get qualitatively?

2. We can interpret the smoothing parameter as adding a constant number of occurrences of each word to our distribution.  Does this help you determine set a reasonable value for this 'prior factor'?

3. For fun: also check out [Amazon's Statistically Improbable Phrases](http://en.wikipedia.org/wiki/Statistically_Improbable_Phrases).

*Implementation note:*
As you adjust the size of the Bayesian smoothing parameter, you will notice first nonsense phrases being removed and then legitimate bigrams being removed, leaving you with only generic bigrams.  The goal is to find a value of the smoothing parameter between these two transitions.

The reference solution is not an aggressive filterer: it errors in favor of leaving apparently nonsensical words. On further consideration, many of these are actually somewhat meaningful. The smoothing parameter chosen in the reference solution is equivalent to giving each word 90 previous appearances prior to considering this data.  This was chosen by generating a list of bigrams for a range of smoothing parameters and seeing how many of the bigrams were shared between neighboring values.  When the shared fraction reached 95%, we judged the solution to have converged.  Note that `min_df` should not be set too high, where it could exclude these borderline words.

In [27]:
import scipy.sparse
from collections import defaultdict
import dill
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from grader import score
import heapq
from random import randint

In [28]:
restaurant_reviews = dill.load(open("Rest_Reviews.pkl", "r")) #try first 100,000 of those

In [166]:
restaurant_reviews

[u"A really lovely surprise on a rather horrific family road trip.\n\nWe were greeted at the door, the waitress was a peach, and the nice man who filled water/helped out was the sweetest guy ever. The food is also great if you're a fan of the diner-type grub; my hot turkey sandwich will live on in my dreams.\n\nYou can definitely tell the place has the old time appeal of a great family place as all the locals (mostly older people) know each other and it's clear it's the happening place for breakfast with the over-60 crowd (but hey, that says the food has to be pretty homestyle and tasty to me.)  \n\nIf you're headed down 94 and need a good lunch stop, make this it.",
 u'Very nice and clean place to have breakfast or lunch',
 u'This place is pretty dumpy - "Corinthian leather" chairs, crappy tables...\nWhat is worth going for is the walleye and eggs served on weekends.  It is a HUGE portion of walleye, lighty fried, served with eggs cooked to order, potatoes and toast.  All this for som

In [148]:
rand_param = 5
# tuned this -- try gridsearch?? or something like it. See video  (siraj hyperparameters on youtube- slack)

In [149]:
revs = [x for x in restaurant_reviews if randint(1,10) <= rand_param]

In [150]:
print len(revs),",",  len(restaurant_reviews)

28513 , 57263


In [151]:
review_counter = CountVectorizer(ngram_range=(1,2), min_df = .00005)

In [152]:
count_matrix = review_counter.fit_transform(revs)

In [153]:
feature_list = list(review_counter.get_feature_names())

In [154]:
def using_coo(x, features):
    count = defaultdict(lambda: 0)
    cx = scipy.sparse.coo_matrix(x)    
    for i,j,v in tqdm_notebook(zip(cx.row, cx.col, cx.data)):
        count[features[j]] += v
    return count

In [155]:
counts = using_coo(count_matrix, feature_list)

HBox(children=(IntProgress(value=0, max=5010589), HTML(value=u'')))

In [156]:
bigram_counts = defaultdict(lambda: 0)
unigram_counts = defaultdict(lambda: 0)

In [157]:
for key, value in tqdm_notebook(counts.iteritems()):
    keysplit = key.split(" ")
    if len(keysplit) > 1:
        bigram_counts[key] = value
    else:
        unigram_counts[key] = value

HBox(children=(IntProgress(value=1, bar_style=u'info', max=1), HTML(value=u'')))

In [158]:
def calc_stat2(bg_ct, wi_ct, wj_ct, alpha):
    num = bg_ct
    den = (wi_ct + alpha) * (wj_ct + alpha)
    return (num*1.0)/den

In [159]:
dubs = {}
smoothing = 65
n_counts = len(counts)
for key, value in tqdm_notebook(counts.iteritems()):
    try:
        cut = key.split(" ")
    except AttributeError:
        pass
    if len(cut) > 1:
            dubs[key] = calc_stat2(value, counts[cut[0]], counts[cut[1]], smoothing)

HBox(children=(IntProgress(value=1, bar_style=u'info', max=1), HTML(value=u'')))

In [160]:
len(dubs)

223905

In [161]:
foo = sorted(dubs.items(), key = lambda x: x[1], reverse=True)[:100]

In [162]:
dub_words = [x[0] for x in foo]

In [165]:
dub_words

100

In [164]:
score('nlp__food_bigrams', lambda: dub_words)

Your score:  0.2


In [48]:
#score('nlp__food_bigrams', lambda: ["kare kare"] * 100)

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*